<a href="https://colab.research.google.com/github/POS-Cross/Bravo-EDA/blob/main/ML/Bravo_Recommendation_basd_on_frequently_bought_togetherV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d khaledanaqwa/bravodata

 96% 108M/112M [00:00<00:00, 108MB/s]  
100% 112M/112M [00:00<00:00, 136MB/s]


In [ ]:
import pandas as pd

CSV = '/content/bravodata.zip'

df_items=pd.read_csv(CSV)
df_items

,dTicketInternalKey,BravoNo,Invoice_Date,cBrutoValue,POS_NO,BillNo,dItemInternalKey,Barcode,ItemName,GrossRetailPrice,Quantity,TotalBeforeDisc,DiscValue,NetSales_IncVAT,Total_Net_InvoiceIncVAT,bHasPromotions,OfferNo,dVendorId,DeptNo,CashierNo,CustomerGroup,CustomerNo
0,30860300,1,01/09/2019 0:00:00,11.75,2,50200,142,121017,Tnuva Amek Cheese,48.0,0.24,11.75,1.95,9.80,18.8,-1,13369,103260.0,104,804,0,0
1,30860300,1,01/09/2019 0:00:00,6.00,2,50200,21718,252164,Healthy Bread Pcs,6.0,1.00,6.00,0.00,6.00,18.8,0,0,104540.0,107,804,0,0
2,30860300,1,01/09/2019 0:00:00,2.00,2,50200,75254,7622210622211,Cadbury 5Star 48G,2.0,1.00,2.00,0.00,2.00,18.8,0,0,106470.0,112,804,0,0
3,30860300,1,01/09/2019 0:00:00,1.00,2,50200,77885,6221007027751,Nesquik Biscuit Milk 56G,1.0,1.00,1.00,0.00,1.00,18.8,0,0,101520.0,112,804,0,0
4,30860301,1,01/09/2019 0:00:00,25.00,2,50201,1292,7622100919261,Marlboro Red,25.0,1.00,25.00,0.00,25.00,45.0,0,0,501024.0,108,804,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481787,31755806,1,14/12/2019 0:00:00,30.00,5,38929,63798,8906021075107,Peanut Butter Ruparels Red 510G,15.0,2.00,30.00,10.00,20.00,268.9,-1,15967,100405.0,112,847,1,628959
3481788,31755806,1,14/12/2019 0:00:00,20.00,5,38929,68245,5449000127631,Jericho Water 8*1.5L Case,10.0,2.00,20.00,3.00,17.00,268.9,-1,16124,101730.0,112,847,1,628959
3481789,31755806,1,14/12/2019 0:00:00,22.00,5,38929,69643,7290001996553,Alaa Al Deen Tissues 800G 2P,22.0,1.00,22.00,2.00,20.00,268.9,-1,14084,101590.0,108,847,1,628959
3481790,31755806,1,14/12/2019 0:00:00,40.00,5,38929,76574,6253504290055,Reema Toilet 32 Roll,40.0,1.00,40.00,8.00,32.00,268.9,-1,15482,101590.0,108,847,1,628959


In [ ]:
df_items.drop_duplicates("dTicketInternalKey").Total_Net_InvoiceIncVAT.sum()

40922634.68

In [ ]:
df_items.dItemInternalKey.value_counts()

69429    32758
60572    29732
60578    28655
60574    24841
60576    23667
         ...  
70975        1
62726        1
72322        1
67359        1
70707        1
Name: dItemInternalKey, Length: 9380, dtype: int64

In [ ]:
products = df_items[['dItemInternalKey', 'ItemName']].copy()
products = products[~products.duplicated()]
products



,dItemInternalKey,ItemName
0,142,Tnuva Amek Cheese
1,21718,Healthy Bread Pcs
2,75254,Cadbury 5Star 48G
3,77885,Nesquik Biscuit Milk 56G
4,1292,Marlboro Red
...,...,...
3469260,19574,Cheese 600G Pasteurised
3469989,78303,Post Oreo Golden Cereal 311G
3471232,63789,Gullon Mini Cracker Biscuts 350G
3478939,56722,Cake 50 Prs


In [ ]:
# Set the index to StockCode
products = products.set_index('dItemInternalKey')

# Convert to Series for eve easier lookups
products = products['ItemName']
products

dItemInternalKey
142      Tnuva Amek Cheese                       
21718    Healthy Bread Pcs                       
75254    Cadbury 5Star 48G                       
77885    Nesquik Biscuit Milk 56G                
1292     Marlboro Red                            
                           ...                   
19574    Cheese 600G Pasteurised                 
78303    Post Oreo Golden Cereal 311G            
63789    Gullon Mini Cracker Biscuts 350G        
56722    Cake 50 Prs                             
53024    Karmel 11.99                            
Name: ItemName, Length: 9380, dtype: object

In [ ]:
df_items = df_items.dropna()

orders=df_items[['dTicketInternalKey','dItemInternalKey']]

def string_list(x):
    return [str(i) for i in x]

orders = orders.groupby('dTicketInternalKey')['dItemInternalKey'].apply(list).reset_index()
orders.head()

,dTicketInternalKey,dItemInternalKey
0,30860300,"[142, 21718, 75254, 77885]"
1,30860301,"[1292, 21524]"
2,30860302,[10952]
3,30860303,"[4291, 77403]"
4,30860304,"[4166, 68248]"


In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()

te.fit(orders['dItemInternalKey'])
orders_1hot = te.transform(orders['dItemInternalKey'])
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
orders_1hot

,1,71,74,76,77,78,79,80,81,127,128,139,140,141,142,143,145,146,148,149,251,255,256,257,281,294,296,297,298,302,303,304,305,306,307,308,310,312,313,321,...,78254,78255,78258,78260,78261,78262,78264,78265,78266,78267,78268,78269,78272,78273,78274,78275,78276,78278,78279,78280,78281,78288,78289,78290,78291,78292,78293,78294,78298,78301,78303,78308,78309,78310,78315,78317,78318,78319,78323,78324
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717988,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
717989,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [ ]:
! pip install mlxtend --upgrade


In [ ]:
from mlxtend.frequent_patterns import fpgrowth
%timeit -n1 -r1

fpgrowth(orders_1hot, min_support=0.01, max_len=2, use_colnames=True)




,support,itemsets
0,0.032119,(142)
1,0.019964,(1292)
2,0.012499,(343)
3,0.019876,(19244)
4,0.014709,(54837)
5,0.025415,(11027)
6,0.011134,(73480)
7,0.019951,(60568)
8,0.041410,(60572)
9,0.017893,(9687)


In [ ]:
is_fp = fpgrowth(orders_1hot, min_support=0.001, max_len=3, use_colnames=True)
is_fp

,support,itemsets
0,0.032119,(142)
1,0.002185,(75254)
2,0.019964,(1292)
3,0.004657,(21524)
4,0.002415,(10952)
...,...,...
1599,0.001446,"(15530, 60572)"
1600,0.001079,"(60568, 15530)"
1601,0.001334,"(15530, 60574)"
1602,0.001077,"(60588, 60572)"


In [ ]:
len(is_fp)

1604

In [ ]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(is_fp, metric="lift", min_threshold=10)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(60580),(60582),0.006538,0.025241,0.001747,0.267150,10.583872,0.001582,1.330092
1,(60582),(60580),0.025241,0.006538,0.001747,0.069194,10.583872,0.001582,1.067314
2,(60580),(60583),0.006538,0.014815,0.001304,0.199403,13.459651,0.001207,1.230564
3,(60583),(60580),0.014815,0.006538,0.001304,0.087995,13.459651,0.001207,1.089316
4,"(60578, 60572)",(60580),0.013128,0.006538,0.001251,0.095268,14.572230,0.001165,1.098074


In [ ]:
def predict(antecedent, rules, max_results= 6):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a messy way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results].reset_index(drop=True)

In [ ]:
preds = predict({15530}, rules)
preds

0    60568
Name: consequents, dtype: int64

In [ ]:
for stockid in preds:  
    print(products[stockid])

Produce 3.33 Nis                        


In [ ]:
itemset ={1748}
# get the predictions
preds = predict(itemset, rules)
print(products[1748])
# Display the descriptions of the predictions
for stockid in preds:  
    print(products[stockid])

Roast Beef                              


In [ ]:
products[25206]

'Snickers 50G                            '

In [ ]:
data = {}
for item in products.index:  
  preds = predict({item}, rules)
  arr=[]
  for stockid in preds:  
    # print(products[stockid])
    arr.append(products[stockid])
  data[item]=arr
data

{142: ['Siniora Turkey Breast Pabrika           ',
  'Siniora Bbq                             ',
  'Mays Hodlavan Turkey Breast             ',
  'Oaf Tof Breast And Basterma             ',
  'Salami Oaf Tof                          ',
  'Sin Roast Beef Plain                    '],
 21718: [],
 75254: [],
 77885: [],
 1292: [],
 21524: [],
 10952: [],
 4291: [],
 77403: [],
 4166: [],
 68248: [],
 55884: [],
 60563: [],
 343: [],
 64260: [],
 13867: [],
 77022: [],
 56896: [],
 73662: [],
 60580: ['Produce 10 Nis                          ',
  'Produce 12 Nis                          ',
  'Produce 8 Nis                           ',
  'Produce 5 Nis                           ',
  'Produce 8 Nis                           ',
  'Produce 7 Nis                           '],
 60603: [],
 63361: [],
 73310: [],
 19244: [],
 75955: [],
 77925: [],
 77717: [],
 54837: ['Baby Wipes Purple                       ',
  'Baby Wipes Tango                        ',
  'Baby Wipes Tango                     